In [34]:
import pandas as pd
import numpy as np

# Naive Bayes Classifier 
## Numpy Implementation

In [271]:
def get_results(x_input, X, Y):
    return {j: (
                (sum(Y==j) / Y.shape[0]) 
                * np.prod( [sum(( X[:,i] == x_input[i] ) & (Y==j)) / sum(Y==j)
                            for i in range(X.shape[1])
                           ])
                ) for j in np.unique(Y)
           }

def get_outcome(results):
    return max(results, key=results.get)

In [272]:
%%timeit
X = np.array([['urgent', 'y', 'y'],
       ['urgent', 'n', 'y'],
       ['near', 'y', 'y'],
       ['none', 'y', 'n'],
       ['none', 'n', 'y'],
       ['none', 'y', 'n'],
       ['near', 'n', 'n'],
       ['near', 'n', 'y'],
       ['near', 'y', 'y'],
       ['urgent', 'n', 'n']])
Y = np.array(['party', 'study', 'party', 
           'party', 'pub', 'party', 
           'study', 'tv', 'party', 
           'study'])
x_input = ['near', 'n', 'y']

get_outcome(get_results(x_input,X,Y))

1.16 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Pandas Implementation

In [273]:
def prob(y, Y):
    return Y.value_counts().loc[y] / Y.shape[0]
def cond_prob(x, X, y, Y):
    return sum((X==x)&(Y==y)) / Y.value_counts().loc[y]
def mult_cond_prob(df, X, y, Y):
    
    return np.prod([cond_prob(v, df[k], y, df[Y]) 
                    for k,v in X.items()]
                  ) * prob(y, df[Y])
    
def all_outcomes(df, X, Y):
    outcomes = df[Y].unique()
    probs = {}
    for o in outcomes:
        probs[o] = mult_cond_prob(df, X, o, Y)
    return probs    

In [274]:
%%timeit
df = pd.DataFrame({'deadline':['urgent','urgent','near','none','none','none',
                               'near','near','near','urgent'],
                   'party':['y','n','y','y','n','y','n','n','y','n'],
                   'lazy':['y','y','y','n','y','n','n','y','y','n'],
                   'activity':['party','study','party','party','pub','party','study','tv','party','study']
                  })
all_outcomes(df, {'deadline':'near','party':'n','lazy':'y'}, 'activity')

29.4 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
